Thoushands or even millions of features not only make training extremely slow, also make it harder to find a good solution, this is called **the curse of dimensionality**.

# Main content
- the curse of dimensionality
- know high-dimensional space
- projection and Manifold learning
- PCA, Kernel PCA, LLE.

# The Curse of Dimensionality
The more dimensions the training set has, the greater the risk of overfitting it. In theory, one solution to the curse of dimensionality could be to increase the size of training set to reach a sufficient density of training instances.

# Main Approaches for Dimensionality Reduction
## Projection
Training instances are not spread out uniformly across all dimensions. Many features are almost constant, while others are highly correlated. As a result, all training instances actually lie within a much lower-dimensional space. See an example.

![8](images/8-2.png)

![8](images/8-3.png)

![8](images/8-4.png)

## Manifold Learning

![8](images/8-5.png)

![8](images/8-6.png)

# PCA
Principal Component Analysis is the most popular dimensionality reduction algorithm. First it identifies the hyperplane that lies closest to the data, and then it projects the data onto it.

## Preserving the Variance
Before projecting the training set onto a lower-dimensional htperplane, you first need to choose the right hyperplane.

![8](images/8-7.png)

## Principal Components
**PCA identifies the axis that accounts for the largest amount of variance in the training set.**

The unit vector that defines the $i^{th}$ axis is called the $i^{th}$ **principal component(PC)**. In fig 8-7, the 1st PC is $c_1$ and the 2nd PC is $c_2$.

### How to find the PC of a training set?
There is a standard matrix factorization technique called **Singular Value Decomposition(SVD)** that can decompose the training set matrix **X** into the ot product of three matrices $U\cdot \sum \cdot V^T$, where $V^T$ contains all the principal components that we are looking for.

![8](images/e8-1.png)

Use Numpy's `svd()` function to obtain all the principal components of the training set, then extracts the first two PCs:
```
X_centered = X - X.mean(axis=0)
U,s,V = np.linalg.svd(X_centered)
c1 = V.T[:,0]
c2 = V.T[:,1]
```

**PCA assumes that the dataset is centered around the origin. Don't forget to center the data first**.

## Projecting Down to d Dimensions
Once you have identified all the principal components, you can reduce the dimensionality of the dataset down to d dimensions by projecting it onto the hyperplane defined by the first d principal components. Selecting this hyperplane ensures that the projection will preserve as much variance as possible.

$W_d$ is the matrix containing the first d principal components(i.e., the matrix composed of the first d columns of $V^T$.

![8](images/e8-2.png)

## Using Scikit-Learn
```
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

first_PC = pca.components_.T[:,0]
```

## Explained Variance Ratio
Another very useful piece of information is the **explained variance ratio** of each pricipal component. It indicates the proportion of the dataset's variance that lies along the axis of each principal component.

## Choosing the Right Number of Dimensions
Instead of arbitrarily choosing the number of dimensions to reduce down to, it is generally preferable to choose the number of dimensions that add up to a sufficiently large portion of the variance(e.g., 95%). 

```
pca=PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >=0.95)+1
```

And a better option:
```
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)
```

![8](images/8-8.png)

## PCA for Compression

![8](images/8-9.png)

![8](images/e8-3.png)

## Incremental PCA
**One problem with the preceding implementation of PCA is that it requires the whole training set to fit in memory in order for the SVD algorithm to run**.

**Incremental PCA(IPCA) algorithms:Split the training set into mini-batches and feed an IPCA one mini batch at a time.**

- First implemention

```
from sklearn.decomposition import IncrementalPCA
n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_mnist, n_batches):
    inc_pca.partial_fit(X_batch)
    
X_mnist_reduced = inc_pca.transform(X_mnist)
```


- Second implemention. Numpy's `memmap` class, which allows you to manipulate a large array stored in a binary file on disk as if it were entirely in memory;the class loads only the data it needs in memory, when it needs it.

```
X_mm = np.memmap(filename, dtype='float32', mode='readonly', shape=(m,n))
batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)
```


## Randomized PCA
It is a stochastic algorithm that quickly finds an approximation of the first d principal components. It is dramatically faster than the previous algorithms when d is much smaller than n.

```
rnd_pca = IncrementalPCA(n_components=154, svd_solver='randomized')
X_reduced = rnd_pca.fit_transform(X_mnist)
```

# Kernel PCA(kPCA)
It is often good at preserving clusters of instances after projection, or sometimes even unrolling datasets that lie close to a twisted manifold.

```
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components=2, kernel='rbf', gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)
```

![8](images/8-10.png)

# Selecting a Kernel and Tuning Hyperparameters
Dimensionality reduction is often a preparation step for a supervised learning task, so we can use grid search to select the kernel and hyperparameters that lead to the best performance on that task.

For example, the following code creates a two-step pipeline, first reducing dimensionality to two dimensions using kPCA, then applying Logistic Regression for classification. Then it uses Grid SearchCV to find the best kernel and gamma value for kPCA in order to get the best classification accuracy at the end of the pipeline.

![8](images/c8-2.png)

